In [ ]:
!pip install google-generativeai
!pip install langchain
!pip install langchain_google_genai
!pip install python-dotenv
!pip install langchain_community
!pip install wikipedia
!pip install "langchain-unstructured[local]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=9265b69eaa810b1c84d3ebd4cb8cfcfa90e7cfdaecd982c14f8f8197eada2c23
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.7 MB

In [ ]:
!pip install langchain_chroma

In [ ]:
from google.colab import userdata
api_key = userdata.get('GOOGLEAPIKEY')

KeyboardInterrupt: 

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
    GoogleGenerativeAI
)
import os
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

In [ ]:
import os
# from dotenv import load_dotenv

# load_dotenv()
os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
llm = GoogleGenerativeAI(
	model="models/gemini-1.5-pro-exp-0827",
	temperature=0.5,
	google_api_key= api_key,
	safety_settings={
		HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
		HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
		HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE}

	)


In [ ]:
llm.invoke("What company developed you")

In [ ]:
url = "https://docs.manim.community/en/stable/tutorials/quickstart.html"

# RAG

In [ ]:
from langchain_unstructured import UnstructuredLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=(url,),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("<p>")
    #     )
    # ),
)
docs = loader.load()




In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
urls = []
pages = []
visited = {""}
# function add_page(url):
#     pass

def all_hrefs(href):
    return href and not "#" in href and href.startswith("reference")

def url_to_text(url):
    try:
      fp = urllib.request.urlopen(url)
    except:
      return
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    soup = BeautifulSoup(mystr)
    for code in soup.find_all(class_="pre"):
        if code.string:
          code.string.replace_with("`"+code.string+"`")
          if "[source]" in code.string:
              code.string.replace_with("")

    for code in soup.find_all("pre"):
        if code.string:
          code.decompose()

    for code in soup.find_all(class_="highlight"):
        for pre in code.find_all("pre"):
            spans = pre.find_all("span")
            # print(spans)
            first_span = spans[1]
            last_span = spans[-1]
            if first_span.string and last_span.string:
                first_span.string.replace_with("`"+first_span.string)
                last_span.string.replace_with(last_span.string+"`")

    urls.append(url)
    pages.append(soup.article.get_text())

    soup = soup.article

    links = soup.find_all(href=all_hrefs)
    for link in links:
        link["href"] = "https://docs.manim.community/en/stable/" + link["href"]
        if link["href"] not in visited:
            visited.add(link["href"])
            url_to_text(link["href"])

visited.add("https://docs.manim.community/en/stable/reference.html")
url_to_text("https://docs.manim.community/en/stable/reference.html")
print(urls)

['https://docs.manim.community/en/stable/reference.html', 'https://docs.manim.community/en/stable/reference_index/animations.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.Add.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.Animation.html', 'https://docs.manim.community/en/stable/reference/manim.animation.animation.Wait.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.AnimatedBoundary.html', 'https://docs.manim.community/en/stable/reference/manim.animation.changing.TracedPath.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.html', 'https://docs.manim.community/en/stable/reference/manim.animation.composition.AnimationGroup.html', 'https://docs.manim.community/en/stable/reference/manim.an

In [ ]:
len(urls),len(pages)

(191, 191)

In [ ]:
!pip install chromadb

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
# from google.colab import userdata
# from torch


# hf_api = userdata.get("huggingface")
huggingface_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="NovaSearch/stella_en_1.5B_v5",
    device = "cuda"
)
chroma_client = chromadb.PersistentClient(path="./manim")
collection = chroma_client.create_collection(name="manim_docs2801", embedding_function=huggingface_ef)
ids = urls
collection.add(documents=pages, ids=ids)


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r manim /content/drive/MyDrive

# Post Embedding


In [ ]:
!cp -r /content/drive/MyDrive/manim ./


In [ ]:
!pip install chromadb
!pip install xformers

  Using cached xformers-0.0.29.post2-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cusparselt_cu12-0.6.2-py3-none-manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached xformers-0.0.29.post2-cp311-cp311-manylinux_2_28_x86_64.whl (44.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 5.4 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu124
    Uninstalling torch-2.5.1+cu124:
      Successfully uninstalled torch

In [ ]:
!pip install -U torch

In [ ]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.29.post2 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from google.colab import userdata
huggingface_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="NovaSearch/stella_en_400M_v5",
    device = "cuda",
    trust_remote_code=True
)
chroma_client = chromadb.PersistentClient(path="./manim")
collection = chroma_client.get_collection("manim_docs2801")
collection.query(huggingface_ef.embed_with_retries("Given a web search query, retrieve relevant passages that answer the query 'how to draw a circle'") )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Some weights of the model checkpoint at NovaSearch/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc[-5::])
print(splits)
vectorstore1 = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))


[Document(metadata={'source': '/content/leph1ps.pdf', 'page': 11}, page_content='the wavelength of the photon (or electron), we can determine how things behave under different\nconditions, and we can develop instruments which will use these phenomena to our advantage.\nIt has been a pleasure to work for these books at the higher secondary level, along with a\nteam of members. The Textbook Development Team, Review Team and Editing Teams involved\ncollege and university teachers, teachers from Indian Institutes of Technology, scientists from\nnational institutes and laboratories, as well as, higher secondary teachers. The feedback and\ncritical look provided by higher secondary teachers in the various teams are highly laudable.\nMost box items were generated by members of one or the other team, but three of them were\ngenerated by friends and well-wishers not part of any team. W e are thankful to Dr P .N. Sen of\nPune, Professor Roopmanjari Ghosh of Delhi and Dr Rajesh B Khaparde of Mumb

In [ ]:
retriever = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retriever.invoke("what is capacitance")

[Document(metadata={'page': 12, 'source': '/content/leph1ps.pdf'}, page_content='CONTENTS\nFOREWORD v\nPREFACE xi\nCHAPTER ONE\nELECTRIC  CHARGES  AND FIELDS\n1.1 Introduction 1\n1.2 Electric Charge 1\n1.3 Conductors and Insulators 5\n1.4 Charging by Induction 6\n1.5 Basic Properties of Electric Charge 8\n1.6 Coulomb’s Law 10\n1.7 Forces between Multiple Charges 15\n1.8 Electric Field 18\n1.9 Electric Field Lines 23\n1.10 Electric Flux 25\n1.11 Electric Dipole 27\n1.12 Dipole in a Uniform External Field 31\n1.13 Continuous Charge Distribution 32\n1.14 Gauss’s Law 33\n1.15 Applications of Gauss’s Law 37\nCHAPTER TWO\nELECTROSTATIC  POTENTIAL  AND CAPACITANCE\n2.1 Introduction 51\n2.2 Electrostatic Potential 53\n2.3 Potential due to a Point Charge 54\n2.4 Potential due to an Electric Dipole 55\n2.5 Potential due to a System of Charges 57\n2.6 Equipotential Surfaces 60\n2.7 Potential Energy of a System of Charges 61\n2.8 Potential Energy in an External Field 64\n2.9 Electrostatics of Cond

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)[::3]
vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))


In [ ]:
from langchain_unstructured import UnstructuredLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader

loaderpdf = PyPDFLoader("/content/leph1ps.pdf")

doc = loaderpdf.load()

In [ ]:
ret = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever| format_docs "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is ?")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-38-46186de4fa24>, line 19)

# RAG via google api (vanilla)

In [ ]:
from google.genai import genai
